<a href="https://colab.research.google.com/github/aram-koorn/meridian/blob/main/mmm_attribution_release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import pandas as pd
from datetime import datetime

from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [83]:
START_DATE = '2025-07-01'
END_DATE = '2025-08-30'

In [84]:
QUERY = f"""
SELECT
country,
channel,
goal,
SUM(attributed_orders) as orders,
SUM(impressions) as impressions,
SUM(cost) as costs
 FROM
  `coolblue-marketing-prod.sales_marketing_data_model.sales_marketing_data`
  WHERE interaction_date between '{START_DATE}' and '{END_DATE}'
  and channel in ('Video', 'Social', 'Price Comparison', 'Display', 'Google Ads', "Microsoft Ads", 'Affiliate')
  -- and (channel = 'Social' AND goal = 'Sales Marketing')
  and country = 'Germany'
  and goal in ('Partner Marketing', 'Product Sales', 'Coolblue Branding', 'Product Branding', 'Employer Branding')
  group by all
  """

In [85]:
QUERY_OVERVIEW = f"""

 with tmp as (SELECT
channel,
"uat" as env,
SUM(attributed_orders) as orders
 FROM
 `coolblue-marketing-uat.attribution_data.attribution_results_dashboard`
 WHERE interaction_date between '{START_DATE}' and '{END_DATE}'
 and region in ('DE', "DE Other", "NRW")
--  and region in ('DE')
 group by all
 UNION ALL
 SELECT
channel,
"prod" as env,
SUM(attributed_orders) as orders
 FROM
 `coolblue-marketing-prod.attribution_data.attribution_results_dashboard`
 WHERE interaction_date between '{START_DATE}' and '{END_DATE}'
 and region in ('DE', "DE Other", "NRW")
  -- and region in ('DE')
 group by all
 )
,
res_att as (SELECT *
FROM tmp
PIVOT (
  SUM(orders)  -- or MAX/MIN if only one row per env+channel
  FOR env IN ('uat', 'prod')
)
order by channel
)
,
res as (
SELECT
country,
channel,
goal,
SUM(attributed_orders) as orders,
SUM(impressions) as impressions,
SUM(cost) as costs
 FROM
  `coolblue-marketing-prod.sales_marketing_data_model.sales_marketing_data`
  WHERE interaction_date between '{START_DATE}' and '{END_DATE}'
  and channel in ('Video', 'Social', 'Price Comparison', 'Display', 'Google Ads', "Microsoft Ads", 'Affiliate')
  -- and (channel = 'Social' AND goal = 'Sales Marketing')
  and country = 'Germany'
  and goal in ('Partner Marketing', 'Product Sales', 'Coolblue Branding', 'Product Branding', 'Employer Branding')
  group by all
)
,
display as (
  select
  'display' as channel,
  -- channel,
  SUM(impressions) as impressions,
  SUM(costs) as costs
  FROM
  res
  where channel = 'Display'
  and goal = 'Product Sales'
  group by all
)
,
video as (
  select
    'video' as channel,
  -- channel,
  SUM(impressions) as impressions,
  SUM(costs) as costs
  FROM
  res
  where channel = 'Video'
  -- and goal in ('Coolblue Branding', 'Partner Marketing', 'Product Branding', 'Employer Branding')
  and goal in ('Coolblue Branding', 'Partner Marketing', 'Product Branding')
  group by all
)
,
social as (

  select
      'social' as channel,
  -- channel,
  SUM(impressions) as impressions,
  SUM(costs) as costs
  FROM
  res
  where channel = 'Social'
  and goal = 'Product Sales'
  group by all
)
,

google_ads as (
  select
    'google ads' as channel,
  -- channel,
  SUM(impressions) as impressions,
  SUM(costs) as costs
  FROM
  res
  where channel = 'Google Ads'
  and goal in ('Product Sales')
  group by all
)
,
ma as (
  select
    'microsoft ads' as channel,
  -- channel,
  SUM(impressions) as impressions,
  SUM(costs) as costs
  FROM
  res
  where channel = 'Microsoft Ads'
  group by all
)
,

pc as (
  select
    'pricecomparison' as channel,

  -- channel,
  SUM(impressions) as impressions,
  SUM(costs) as costs
  FROM
  res
  where channel = 'Price Comparison'
  group by all
)
,

aff as (
  select
    'affiliate' as channel,

  -- channel,
  SUM(impressions) as impressions,
  SUM(costs) as costs
  FROM
  res
  where channel = 'Price Comparison'
  group by all
)
,
overview AS (

select * from display
UNION ALL
select * from video
UNION ALL
select * from social
UNION ALL
select * from google_ads
UNION ALL
select * from pc
UNION ALL
select * from ma
UNION ALL
SELECT * from aff
)

select
*,
uat - prod as diff
, uat / SUM(uat) OVER () AS uat_order_share
, prod / SUM(prod) OVER () AS prod_order_share
, SAFE_DIVIDE(costs, uat) as  uat_cost_per_order
, SAFE_DIVIDE(costs, prod) as  prod_cost_per_order
, costs / SUM(costs) OVER () as cost_share
-- , costs/ prod AS prod_cost_per_order
from res_att
LEFT JOIN overview
ON overview.channel=res_att.channel
order by res_att.channel
"""

In [86]:
# -------------------------------
# Run query
# -------------------------------
def fetch_data_from_bigquery(query: str, project: str = None) -> pd.DataFrame:
    """
    Execute the SQL query on BigQuery and return
    the results as a pandas DataFrame.
    """
    try:
        client = bigquery.Client(project=project)  # Initialize BigQuery client
        query_job = client.query(query)     # Run the query job
        df = query_job.to_dataframe()    # Convert results to pandas DataFrame
        print(f"Loaded {len(df)} rows from BigQuery.")
        return df
    except Exception as e:
        print(f"Error fetching data from BigQuery: {e}")
        raise

In [87]:
#Run query
data_df = fetch_data_from_bigquery(QUERY, project="coolblue-marketing-dev")

df = data_df.copy()

Loaded 19 rows from BigQuery.


In [88]:
data_overview = fetch_data_from_bigquery(QUERY_OVERVIEW, project="coolblue-marketing-dev")

Loaded 16 rows from BigQuery.


In [89]:
data_overview

,channel,uat,prod,channel_1,impressions,costs,diff,uat_order_share,prod_order_share,uat_cost_per_order,prod_cost_per_order,cost_share
0,affiliate,6715.407369,7748.487921,affiliate,0.000000e+00,2.339051e+05,-1033.080553,0.049215,0.056965,34.831105,30.187188,0.067488
1,direct,37431.798864,41009.762917,None,NaN,NaN,-3577.964053,0.274324,0.301494,NaN,NaN,NaN
2,direct app,3414.515393,3939.349683,None,NaN,NaN,-524.834290,0.025024,0.028961,NaN,NaN,NaN
3,display,3327.442915,0.000000,display,7.126440e+07,3.428730e+04,3327.442915,0.024386,0.000000,10.304400,NaN,0.009893
4,email,363.768973,415.721334,None,NaN,NaN,-51.952361,0.002666,0.003056,NaN,NaN,NaN
5,google ads,38330.611844,38336.313575,google ads,1.762778e+08,2.122141e+06,-5.701731,0.280911,0.281840,55.364129,55.355895,0.612300
6,microsoft ads,1947.263967,2248.904295,microsoft ads,2.047023e+07,7.670456e+04,-301.640328,0.014271,0.016533,39.390941,34.107525,0.022132
7,organic,7794.333925,8977.193921,None,NaN,NaN,-1182.859996,0.057122,0.065998,NaN,NaN,NaN
8,other,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,out of home,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df['percentage_of_total_cost'] = (df['costs'] / df['costs'].sum()) * 100
display(df.head())

,country,channel,goal,orders,impressions,costs,percentage_of_total_cost
0,Germany,Display,Product Branding,0.0,2.728488e+06,8399.213431,0.210977
1,Germany,Social,Coolblue Branding,0.0,4.203099e+06,9308.000000,0.233804
2,Germany,Video,Product Branding,0.0,1.171272e+08,580123.792002,14.571899
3,Germany,Affiliate,Partner Marketing,0.0,0.000000e+00,0.000000,0.000000
4,Germany,Display,Partner Marketing,0.0,1.131283e+07,6034.461847,0.151577


In [91]:
data_overview = fetch_data_from_bigquery(QUERY_OVERVIEW, project="coolblue-marketing-dev")

Loaded 16 rows from BigQuery.


In [92]:
data_overview

,channel,uat,prod,channel_1,impressions,costs,diff,uat_order_share,prod_order_share,uat_cost_per_order,prod_cost_per_order,cost_share
0,affiliate,6715.407369,7748.487921,affiliate,0.000000e+00,2.339051e+05,-1033.080553,0.049215,0.056965,34.831105,30.187188,0.067488
1,direct,37431.798864,41009.762917,None,NaN,NaN,-3577.964053,0.274324,0.301494,NaN,NaN,NaN
2,direct app,3414.515393,3939.349683,None,NaN,NaN,-524.834290,0.025024,0.028961,NaN,NaN,NaN
3,display,3327.442915,0.000000,display,7.126440e+07,3.428730e+04,3327.442915,0.024386,0.000000,10.304400,NaN,0.009893
4,email,363.768973,415.721334,None,NaN,NaN,-51.952361,0.002666,0.003056,NaN,NaN,NaN
5,google ads,38330.611844,38336.313575,google ads,1.762778e+08,2.122141e+06,-5.701731,0.280911,0.281840,55.364129,55.355895,0.612300
6,microsoft ads,1947.263967,2248.904295,microsoft ads,2.047023e+07,7.670456e+04,-301.640328,0.014271,0.016533,39.390941,34.107525,0.022132
7,organic,7794.333925,8977.193921,None,NaN,NaN,-1182.859996,0.057122,0.065998,NaN,NaN,NaN
8,other,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,out of home,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
QUERY_OVERVIEW

'\n\n with tmp as (SELECT\nchannel,\n"uat" as env,\nSUM(attributed_orders) as orders\n FROM\n `coolblue-marketing-uat.attribution_data.attribution_results_dashboard`\n WHERE interaction_date between \'2025-07-01\' and \'2025-08-30\'\n and region in (\'DE\', "DE Other", "NRW")\n--  and region in (\'DE\')\n group by all\n UNION ALL\n SELECT\nchannel,\n"prod" as env,\nSUM(attributed_orders) as orders\n FROM\n `coolblue-marketing-prod.attribution_data.attribution_results_dashboard`\n WHERE interaction_date between \'2025-07-01\' and \'2025-08-30\'\n and region in (\'DE\', "DE Other", "NRW")\n  -- and region in (\'DE\')\n group by all\n )\n,\nres_att as (SELECT *\nFROM tmp\nPIVOT (\n  SUM(orders)  -- or MAX/MIN if only one row per env+channel\n  FOR env IN (\'uat\', \'prod\')\n)\norder by channel\n)\n,\nres as (\nSELECT\ncountry,\nchannel,\ngoal,\nSUM(attributed_orders) as orders,\nSUM(impressions) as impressions,\nSUM(cost) as costs\n FROM\n  `coolblue-marketing-prod.sales_marketing_data_m

In [94]:
print(data_overview['channel'].unique())

['affiliate' 'direct' 'direct app' 'display' 'email' 'google ads'
 'microsoft ads' 'organic' 'other' 'out of home' 'pricecomparison'
 'push notification' 'referral' 'social' 'video' 'voucher']


In [95]:
# --- Imports ---
import pandas as pd
import altair as alt

# Use default theme (latimes adds frames)
alt.themes.enable('default')

# --- Your categorization logic ---
# Reusing the categorization logic from cell hcQwMTz4urlD
def categorize_goal_for_mmm(row):
    channel = row['channel']
    goal = row['goal']
    if channel in ('Display', 'Social'):
        return 'Included in MMM' if goal == 'Product Sales' else 'Not Included in MMM'
    return 'Included in MMM'

df['mmm_goal_group'] = df.apply(categorize_goal_for_mmm, axis=1)

# Channel highlight grouping
# Reusing the channel highlighting logic from cell hcQwMTz4urlD
def categorize_channel_for_highlight(channel):
    return 'MMM Channels' if channel in ['Video', 'Display', 'Social'] else 'Other Channels'

df['channel_group'] = df['channel'].apply(categorize_channel_for_highlight)

# Palettes
# Define a coolblue color palette for the goals
coolblue_goal_palette = ['#285dab', '#0090e3', '#cce9f9', '#F2F7FC', '#999999', '#ff7e00', '#a6cee3'] # Added more colors for goals


# Label strip data & order
# Reusing label strip data and order from cell hcQwMTz4urlD
labels_df = df[['channel', 'channel_group']].drop_duplicates()
channel_order = sorted(list(labels_df['channel'].unique()))
print(channel_order)

# --- Main stacked bars (no axis labels/ticks/domains/grids) ---
# Use the original df DataFrame and color by goal
bars = (
    alt.Chart(df) # Use the original df
      .mark_bar()
      .encode(
          x=alt.X('channel:N',
                  sort=channel_order,
                  axis=alt.Axis(labels=False, ticks=False, title='', domain=False, grid=False)),
          y=alt.Y('costs:Q',
                  title='Costs',
                  stack='zero',
                  axis=alt.Axis(labels=True, ticks=False, domain=False, grid=False)),
          color=alt.Color('goal:N', # Color by the original goal
                          scale=alt.Scale(range=coolblue_goal_palette), # Use coolblue palette for goals
                          title='Goal'),
          tooltip=['channel:N', 'goal:N', 'costs:Q', 'channel_group:N'] # Update tooltip
      )
      .properties(width=600, height=360, title='Costs per Channel') # Update title
      .interactive()
)

# --- Channel Labels with Conditional Color ---
xlabels = (
    alt.Chart(labels_df)
      .mark_text(dy=12, fontWeight='bold')
      .encode(
          x=alt.X('channel:N', sort=channel_order, axis=None),
          y=alt.value(0), # Position at the bottom of the chart
          text='channel:N',
          color=alt.condition(
              alt.datum.channel_group == 'MMM Channels',
              alt.value('#ff6600'),  # Coolblue blue for MMM channels
              alt.value('black')   # Black for other channels
          )
      )
      .properties(width=600, height=40) # Adjust height to accommodate labels
)

# --- Compose & purge ALL frames/lines globally ---
chart = (
    alt.vconcat(bars, xlabels, spacing=0)
      .resolve_scale(x='shared')
      .configure_view(stroke=None)            # removes outer plot box (the “square”)
      .configure_axis(grid=False, domain=False, ticks=False)  # belt-and-suspenders
)

chart.display()

['Affiliate', 'Display', 'Google Ads', 'Microsoft Ads', 'Price Comparison', 'Social', 'Video']


alt.VConcatChart(...)

In [96]:
# --- Imports ---
import pandas as pd
import altair as alt

# Use default theme (latimes adds frames)
alt.themes.enable('default')

# --- Your categorization logic ---
def categorize_goal_for_mmm(row):
    channel = row['channel']
    goal = row['goal']
    if channel in ('Display', 'Social'):
        return 'Included in MMM' if goal == 'Product Sales' else 'Not Included in MMM'
    return 'Included in MMM'

df['mmm_goal_group'] = df.apply(categorize_goal_for_mmm, axis=1)

# Aggregate
df_aggregated = (
    df.groupby(['channel', 'mmm_goal_group'], as_index=False)['costs'].sum()
)

# Channel highlight grouping
def categorize_channel_for_highlight(channel):
    return 'MMM Channels' if channel in ['Video', 'Display', 'Social'] else 'Other Channels'

df_aggregated['channel_group'] = df_aggregated['channel'].apply(categorize_channel_for_highlight)

# Palettes
mmm_palette = ['#0090e3', '#999999']

# Label strip data & order
labels_df = df_aggregated[['channel', 'channel_group']].drop_duplicates()
channel_order = list(labels_df['channel'].unique())

# --- Main stacked bars (no axis labels/ticks/domains/grids) ---
bars = (
    alt.Chart(df_aggregated)
      .mark_bar()
      .encode(
          x=alt.X('channel:N',
                  sort=channel_order,
                  axis=alt.Axis(labels=False, ticks=False, title='', domain=False, grid=False)),
          y=alt.Y('costs:Q',
                  title='Costs',
                  stack='zero',
                  axis=alt.Axis(labels=True, ticks=False, domain=False, grid=False)),
          color=alt.Color('mmm_goal_group:N',
                          scale=alt.Scale(range=mmm_palette),
                          title='MMM goal group'),
          tooltip=['channel:N', 'mmm_goal_group:N', 'costs:Q', 'channel_group:N']
      )
      .properties(width=600, height=360, title='Costs per Channel by MMM Goal Group (Stacked)')
      .interactive()
)

xlabels = (
    alt.Chart(labels_df)
      .mark_text(dy=12, fontWeight='bold')   # 👈 add fontWeight here
      .encode(
          x=alt.X('channel:N', sort=channel_order, axis=None),
          y=alt.value(0),
          text='channel:N',
          color=alt.condition(
              alt.datum.channel_group == 'MMM Channels',
              alt.value('#ff6600'),         # or '#0090e3'
              alt.value('black')
          )
      )
      .properties(width=600, height=40)
)

# --- Compose & purge ALL frames/lines globally ---
chart = (
    alt.vconcat(bars, xlabels, spacing=0)
      .resolve_scale(x='shared')
      .configure_view(stroke=None)            # removes outer plot box (the “square”)
      .configure_axis(grid=False, domain=False, ticks=False)  # belt-and-suspenders
)

chart.display()

alt.VConcatChart(...)

In [97]:
# --- Imports ---
import pandas as pd
import altair as alt

# Use default theme (latimes adds frames)
alt.themes.enable('default')

# --- Your categorization logic ---
def categorize_goal_for_mmm(row):
    channel = row['channel']
    goal = row['goal']
    if channel in ('Display', 'Social'):
        return 'Included in MMM' if goal == 'Product Sales' else 'Not Included in MMM'
    return 'Included in MMM'

df['mmm_goal_group'] = df.apply(categorize_goal_for_mmm, axis=1)

# Aggregate
df_aggregated = (
    df.groupby(['channel', 'mmm_goal_group'], as_index=False)['costs'].sum()
)

# Channel highlight grouping
def categorize_channel_for_highlight(channel):
    return 'MMM Channels' if channel in ['Video', 'Display', 'Social'] else 'Other Channels'

df_aggregated['channel_group'] = df_aggregated['channel'].apply(categorize_channel_for_highlight)

# Palettes
mmm_palette = ['#0090e3', '#999999']

# Label strip data & order
labels_df = df_aggregated[['channel', 'channel_group']].drop_duplicates()
channel_order = list(labels_df['channel'].unique())

# --- Main stacked bars (no axis labels/ticks/domains/grids) ---
bars = (
    alt.Chart(df_aggregated)
      .mark_bar()
      .encode(
          x=alt.X('channel:N',
                  sort=channel_order,
                  axis=alt.Axis(labels=False, ticks=False, title='', domain=False, grid=False)),
          y=alt.Y('costs:Q',
                  title='Costs',
                  stack='zero',
                  axis=alt.Axis(labels=True, ticks=False, domain=False, grid=False)),
          color=alt.Color('mmm_goal_group:N',
                          scale=alt.Scale(range=mmm_palette),
                          title='MMM goal group'),
          tooltip=['channel:N', 'mmm_goal_group:N', 'costs:Q', 'channel_group:N']
      )
      .properties(width=600, height=360, title='Costs per Channel by MMM Goal Group (Stacked)')
      .interactive()
)

xlabels = (
    alt.Chart(labels_df)
      .mark_text(dy=12, fontWeight='bold')   # 👈 add fontWeight here
      .encode(
          x=alt.X('channel:N', sort=channel_order, axis=None),
          y=alt.value(0),
          text='channel:N',
          color=alt.condition(
              alt.datum.channel_group == 'MMM Channels',
              alt.value('#ff6600'),         # or '#0090e3'
              alt.value('black')
          )
      )
      .properties(width=600, height=40)
)

# --- Compose & purge ALL frames/lines globally ---
chart = (
    alt.vconcat(bars, xlabels, spacing=0)
      .resolve_scale(x='shared')
      .configure_view(stroke=None)            # removes outer plot box (the “square”)
      .configure_axis(grid=False, domain=False, ticks=False)  # belt-and-suspenders
)

chart.display()

alt.VConcatChart(...)

In [98]:
import altair as alt
import pandas as pd
import numpy as np
y = 'uat_order_share'
# Reshape the data_overview DataFrame
# Select relevant columns and melt 'uat_order_share' and 'cost_share' into a single 'Value' column
data_overview_melted = data_overview.melt(
    id_vars=['channel'], # Keep channel as identifier
    value_vars=['uat_order_share', 'cost_share'], # Columns to melt
    var_name='Metric', # New column for melted variable names
    value_name='Value' # New column for melted values
).dropna(subset=['Value']) # Drop rows with NaN 'Value' to avoid errors with text marks


# Define coolblue colors for the metrics (assuming blue for order share, orange for cost share)
metric_colors = alt.Scale(domain=[y, 'cost_share'], range=['#0090e3', '#ff7e00']) # Blue for order share, orange for cost share

# Set a theme
alt.themes.enable('latimes')

# Define y-axis tick values at every 10%
y_axis_ticks = [i/10 for i in range(11)] # Generates [0.0, 0.1, 0.2, ..., 1.0]

# Define the base encoding for both bars and text
base_encoding = {
    'x': alt.X('Metric', axis=None), # Hide x-axis for the metric within each channel
    'y': alt.Y('Value', title='Share', axis=alt.Axis(format='%', grid=False, labelFontSize=20, values=y_axis_ticks)), # Single y-axis for the values, formatted as percentage, remove grid, increase label font size, set specific tick values
}

# Create the bar chart with color encoding
bars = alt.Chart(data_overview_melted).mark_bar().encode(
    **base_encoding,
    color=alt.Color(
        'Metric',
        scale=metric_colors,
        legend=alt.Legend(
            title="", # Update legend title
            titleFontSize=20, # Increase legend title font size
            labelFontSize=20, # Increase legend label font size
            # Use labelExpr to relabel legend entries
            labelExpr="{'uat_order_share': 'Order Share', 'cost_share': 'Cost Share'}[datum.label]"
        )
    ) # Color by Metric with specified colors and updated legend title
)

# Create the text layer for annotations
text = alt.Chart(data_overview_melted).mark_text(
    align='center',
    baseline='bottom',
    dy=-5, # Nudge text slightly above the bar
    color='black',
    size=14# Set text color in mark properties,
).encode(
    x=alt.X('Metric', axis=None), # Hide x-axis for the metric within each channel
    y=alt.Y('Value', stack='zero'), # Position text at the top of the bar
    text=alt.Text('Value', format='.1%'), # Format text as percentage
)


# Layer the bars and text, then facet by channel
chart = alt.layer(bars, text).properties(
    width=100, # Further reduced width
    height=500 # Further reduced height
).facet(
    column=alt.Column('channel', header=alt.Header(
        title=None, # Remove individual channel titles
        titleOrient="bottom",
        labelOrient="bottom",
        labelPadding=110,
        titlePadding=30,
        labelColor='black',
        titleColor='black',
        labelAngle=-45,
        labelFontSize=20 # Increase label font size
    )) # Configure column header: removed title, increased padding and font size
).properties(
    title='Current Order Share Versus Cost Share' # Apply main title to the faceted chart
)


# Configure the view to remove outer border
chart = chart.configure_view(
    strokeWidth=0
)

# Configure the axes to remove grid lines
chart = chart.configure_axis(
    grid=False
)


chart.display()

alt.FacetChart(...)

In [99]:
import altair as alt
import pandas as pd
import numpy as np
y = 'prod_order_share'
# Reshape the data_overview DataFrame
# Select relevant columns and melt 'uat_order_share' and 'cost_share' into a single 'Value' column
data_overview_melted = data_overview.melt(
    id_vars=['channel'], # Keep channel as identifier
    value_vars=['prod_order_share', 'cost_share'], # Columns to melt
    var_name='Metric', # New column for melted variable names
    value_name='Value' # New column for melted values
).dropna(subset=['Value']) # Drop rows with NaN 'Value' to avoid errors with text marks


# Define coolblue colors for the metrics (assuming blue for order share, orange for cost share)
metric_colors = alt.Scale(domain=[y, 'cost_share'], range=['#0090e3', '#ff7e00']) # Blue for order share, orange for cost share

# Set a theme
alt.themes.enable('latimes')

# Define y-axis tick values at every 10%
y_axis_ticks = [i/10 for i in range(11)] # Generates [0.0, 0.1, 0.2, ..., 1.0]

# Define the base encoding for both bars and text
base_encoding = {
    'x': alt.X('Metric', axis=None), # Hide x-axis for the metric within each channel
    'y': alt.Y('Value', title='Share', axis=alt.Axis(format='%', grid=False, labelFontSize=20, values=y_axis_ticks)), # Single y-axis for the values, formatted as percentage, remove grid, increase label font size, set specific tick values
}

# Create the bar chart with color encoding
bars = alt.Chart(data_overview_melted).mark_bar().encode(
    **base_encoding,
    color=alt.Color(
        'Metric',
        scale=metric_colors,
        legend=alt.Legend(
            title="", # Update legend title
            titleFontSize=20, # Increase legend title font size
            labelFontSize=20, # Increase legend label font size
            # Use labelExpr to relabel legend entries
            labelExpr="{'prod_order_share': 'Order Share', 'cost_share': 'Cost Share'}[datum.label]"
        )
    ) # Color by Metric with specified colors and updated legend title
)

# Create the text layer for annotations
text = alt.Chart(data_overview_melted).mark_text(
    align='center',
    baseline='bottom',
    dy=-5, # Nudge text slightly above the bar
    color='black',
    size=14# Set text color in mark properties,
).encode(
    x=alt.X('Metric', axis=None), # Hide x-axis for the metric within each channel
    y=alt.Y('Value', stack='zero'), # Position text at the top of the bar
    text=alt.Text('Value', format='.1%'), # Format text as percentage
)


# Layer the bars and text, then facet by channel
chart = alt.layer(bars, text).properties(
    width=100, # Further reduced width
    height=500 # Further reduced height
).facet(
    column=alt.Column('channel', header=alt.Header(
        title=None, # Remove individual channel titles
        titleOrient="bottom",
        labelOrient="bottom",
        labelPadding=110,
        titlePadding=30,
        labelColor='black',
        titleColor='black',
        labelAngle=-45,
        labelFontSize=20 # Increase label font size
    )) # Configure column header: removed title, increased padding and font size
).properties(
    title='Current Order Share Versus Cost Share' # Apply main title to the faceted chart
)


# Configure the view to remove outer border
chart = chart.configure_view(
    strokeWidth=0
)

# Configure the axes to remove grid lines
chart = chart.configure_axis(
    grid=False
)


chart.display()

alt.FacetChart(...)

In [100]:
import altair as alt
import pandas as pd
import numpy as np
y = 'prod_cost_per_order'
# Reshape the data_overview DataFrame
# Select relevant columns and melt 'uat_order_share' and 'cost_share' into a single 'Value' column
data_overview_melted = data_overview.melt(
    id_vars=['channel'], # Keep channel as identifier
    value_vars=['prod_cost_per_order', 'uat_cost_per_order'], # Columns to melt
    var_name='Metric', # New column for melted variable names
    value_name='Value' # New column for melted values
).dropna(subset=['Value']) # Drop rows with NaN 'Value' to avoid errors with text marks


# Define coolblue colors for the metrics (assuming blue for order share, orange for cost share)
metric_colors = alt.Scale(domain=["prod_cost_per_order", 'uat_cost_per_order'], range=['#0090e3', '#ff7e00']) # Blue for order share, orange for cost share

# Set a theme
alt.themes.enable('latimes')

# Define the base encoding for both bars and text
base_encoding = {
    'x': alt.X('Metric', axis=None), # Hide x-axis for the metric within each channel
    'y': alt.Y('Value', title='Cost Per Order', axis=alt.Axis(grid=False, labelFontSize=20, values=[20, 40, 60, 80, 100])), # Single y-axis for the values, formatted as percentage, remove grid, increase label font size, set specific tick values
}

# Create the bar chart with color encoding
bars = alt.Chart(data_overview_melted).mark_bar().encode(
    **base_encoding,
    color=alt.Color(
        'Metric',
        scale=metric_colors,
        legend=alt.Legend(
            title="", # Update legend title
            titleFontSize=20, # Increase legend title font size
            labelFontSize=20, # Increase legend label font size
            # Use labelExpr to relabel legend entries
            labelExpr="{'prod_cost_per_order': 'Prod Cost Per Order', 'uat_cost_per_order': 'Uat Cost Per Order'}[datum.label]"
        )
    ) # Color by Metric with specified colors and updated legend title
)

# Create the text layer for annotations
text = alt.Chart(data_overview_melted).mark_text(
    align='center',
    baseline='bottom',
    dy=-5, # Nudge text slightly above the bar
    color='black',
    size=14# Set text color in mark properties,
).encode(
    x=alt.X('Metric', axis=None), # Hide x-axis for the metric within each channel
    y=alt.Y('Value', stack='zero'), # Position text at the top of the bar
    text=alt.Text('Value', format='.2f'), # Format text as percentage
)


# Layer the bars and text, then facet by channel
chart = alt.layer(bars, text).properties(
    width=100, # Further reduced width
    height=500 # Further reduced height
).facet(
    column=alt.Column('channel', header=alt.Header(
        title=None, # Remove individual channel titles
        titleOrient="bottom",
        labelOrient="bottom",
        labelPadding=110,
        titlePadding=30,
        labelColor='black',
        titleColor='black',
        labelAngle=-45,
        labelFontSize=20 # Increase label font size
    )) # Configure column header: removed title, increased padding and font size
).properties(
    title='Impacted Cost per Order' # Apply main title to the faceted chart
)


# Configure the view to remove outer border
chart = chart.configure_view(
    strokeWidth=0
)

# Configure the axes to remove grid lines
chart = chart.configure_axis(
    grid=False
)


chart.display()

alt.FacetChart(...)